In [ ]:
from os import listdir,mkdir
from os.path import isfile, join, isdir,exists
import pandas as pd
import numpy as np
from scipy import stats
import re
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from myplots import roundup, rounddown, find_decimal_fold, percentile_cut_off, rarefaction_calc, rarefaction_plot,draw_correlation_scatter
from matplotlib.ticker import FormatStrFormatter
import cPickle as pickle
from Bio.SeqUtils import GC
import seaborn as sns
import random
from scipy.stats import pearsonr
from skbio.diversity.alpha import shannon, simpson, berger_parker_d

from pop_organize import get_sample_data, get_sample_with_dfs
from SufficientStatistics import *
from MyFunctionsShani import *


In [ ]:
import time
cdate=str(time.strftime("%d%m%Y"))
cdate

# Find public sequences in the PNP cohort

## all public sequences in the cohort:

### generate unique aa lists for each sample in the cohort:

In [ ]:
def gen_uniqueAAdf(sample_name,data_folder):
    file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/%s/uniqueAAdfs/%s' %(data_folder,sample_name) 
    files=[f for f in listdir(uniqueAAdfs_folder) if isfile(join(uniqueAAdfs_folder, f))]


    if sample_name not in files:
        data_folder='TCR_real_data'
        sample_df=pd.read_table("/net/mraid08/export/genie/Lab/Personal/ShaniBAF/%s/SamplesForAnalysis/%s.tsv" %(data_folder,sample_name))
        sample_df=sample_df.rename(columns={'count (templates/reads)':'count (templates)','count (reads)':'count (templates)'})
        sample_df['prod_stat']=np.where(sample_df['sequenceStatus'] == 'In',1,0)
        f={'count (templates)':['sum','count'],'prod_stat':'mean'}
        uniqAA=sample_df.groupby('aminoAcid').agg(f)
        uniqAA=uniqAA.drop(('count (templates)','count'),axis=1)
        uniqAA['Sample']=sample_name
        uniqAA.to_pickle(file1)
    else:
        print('found uniqueAAdf for this sample...')



In [ ]:
data_folder='TCR_real_data'
sample_dfs_folder='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/%s/SamplesForAnalysis' %data_folder
sample_filenames = [f for f in listdir(sample_dfs_folder) if isfile(join(sample_dfs_folder, f))]
sample_filenames=[f.strip('.tsv') for f in sample_filenames]
print len(sample_filenames)


In [ ]:
uniqueAAdfs_folder='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/%s/uniqueAAdfs/' %data_folder
if not isdir(uniqueAAdfs_folder):
    mkdir(uniqueAAdfs_folder)

for n,sample_name in enumerate(sample_filenames):    
        print  n,sample_name
        gen_uniqueAAdf(sample_name,data_folder)

### count how many samples are shared by each sequence:

In [ ]:
#concatenate all uniqueAA lists from all samples:
uniqueAAdfs_folder='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PublicSeqAnalysis/uniqueAAdfs/' 
All434samples_UniqueAAdfs=concat_summarizing_dfs(uniqueAAdfs_folder)

In [ ]:
All434samples_UniqueAAdfs.head()

In [ ]:
#get sub-dataframe to enable merging with counting DF:
All434samples_UniqueAAdfs_toMerge=All434samples_UniqueAAdfs
All434samples_UniqueAAdfs_toMerge.columns = All434samples_UniqueAAdfs_toMerge.columns.get_level_values(0)
All434samples_UniqueAAdfs_toMerge.head()

In [ ]:
# count in how many samples each sequence appears?
countSharedPNP434samples= All434samples_UniqueAAdfs.index.value_counts()

In [ ]:
#save counting to file:
file2='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/%s/PublicSeqAnalysis/countSharedPNP434samples' %data_folder
countSharedPNP434samples.to_pickle(file2)

In [ ]:
# load counting:
file2='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PublicSeqAnalysis/countSharedPNP434samples' 
countSharedPNP434samples=pd.read_pickle(file2)

In [ ]:
# the most shared sequence is shared among 278 samples:
countSharedPNP434samples.head()

In [ ]:
countSharedPNP434samples=pd.DataFrame(countSharedPNP434samples)
countSharedPNP434samples=countSharedPNP434samples.rename(columns={'aminoAcid':'n_shared_samples'})
countSharedPNP434samples.head()

#### calculate the percentage of public sequences in the cohort:

In [ ]:
public=countSharedPNP434samples[countSharedPNP434samples['n_shared_samples']>1]

In [ ]:
print len(public)

In [ ]:
private=countSharedPNP434samples[countSharedPNP434samples['n_shared_samples']==1]

In [ ]:
print len(private)

In [ ]:
# the percent of public sequences in the cohort is 10.3%:
float(len(public)*100)/(len(private)+len(public))

In [ ]:
countSharedPNP434samples['n_shared_samples'].describe()

## Find public sequences allowing 1 aa change:

In [ ]:
#make a list of all uniqueAA sequences in the cohort:
uniqueAAseqList_PNP434=countSharedPNP434samples.index

In [ ]:
seq1=uniqueAAseqList_PNP434[0]
others=list(uniqueAAseqList_PNP434[1:])

In [ ]:
from Bio import pairwise2



In [ ]:
%%time
for seq2 in others:
#     print pairwise2.align.globalxx(seq1, seq)
    seq1len=len(seq1)
    seq2len=len(seq2)
    lenDiff=abs(seq1len-seq2len)
    if lenDiff<2:
        mm=sum(seq1!=seq2 for seq1,seq2 in zip(seq1,seq2))
        if mm<2:
            print seq1,seq2,mm

the number of matched aa seqeunces allowing 1 aa replacement is very high, so I skip this strategy meanwhile

## find shared sequences among lists of top 100 clonal sequences. 

In [ ]:
def findCutoff(countValues):
    if len(countValues)>99:
        for n in range(99,len(countValues)):
            if not countValues[n]==countValues[n-1]:
                break
        
    else:
        n=len(countValues)
    return n
    

In [ ]:
def gen_uniqueAAdf_top100(sample_name,data_folder):
    file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/%s/PublicSeqAnalysis/uniqueAAdfs/%s' %(data_folder,sample_name)
    file2='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/%s/PublicSeqAnalysis/uniqueAAdfstop100/%s' %(data_folder,sample_name)
    uniqueAAdf_top=pd.read_pickle(file1)
    uniqueAAdf_top=uniqueAAdf_top.sort_values(by=('count (templates)','sum'),ascending=False)
    countValues=list(uniqueAAdf_top[('count (templates)','sum')])
    nCut=findCutoff(countValues)
    print (nCut,countValues[nCut])
    uniqueAAdf_top=uniqueAAdf_top[:nCut]
    uniqueAAdf_top.to_pickle(file2)
    
    return uniqueAAdf_top

#     if sample_name not in files:
#         data_folder='TCR_real_data'
#         sample_df=pd.read_table("/net/mraid08/export/genie/Lab/Personal/ShaniBAF/%s/SamplesForAnalysis/%s.tsv" %(data_folder,sample_name))
#         sample_df=sample_df.rename(columns={'count (templates/reads)':'count (templates)','count (reads)':'count (templates)'})
#         sample_df['prod_stat']=np.where(sample_df['sequenceStatus'] == 'In',1,0)
#         f={'count (templates)':['sum','count'],'prod_stat':'mean'}
#         uniqAA=sample_df.groupby('aminoAcid').agg(f)
#         uniqAA=uniqAA.drop(('count (templates)','count'),axis=1)
#         uniqAA['Sample']=sample_name
#         uniqAA.to_pickle(file1)
#     else:
#         print('found uniqueAAdf for this sample...')
    

In [ ]:
for n,sample_name in enumerate(sample_filenames):
        print  n,sample_name
        gen_uniqueAAdf_top100(sample_name,data_folder)

In [ ]:
#concatenate all uniqueAAdf_top100 lists from all samples:
uniqueAAdfs_top100_folder='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PublicSeqAnalysis/uniqueAAdfstop100/'
All434samples_UniqueAAdfs_top100=concat_summarizing_dfs(uniqueAAdfs_top100_folder)

In [ ]:
All434samples_UniqueAAdfs_top100.head()

In [ ]:
# count in how many samples each sequence appears?
countSharedPNP434samples_top100= All434samples_UniqueAAdfs_top100.index.value_counts()

In [ ]:
#save counting to file:
file2='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PublicSeqAnalysis/countSharedPNP434samples_top100' 
countSharedPNP434samples_top100.to_pickle(file2)

In [ ]:
# the most shared sequence is shared among 49 samples:
countSharedPNP434samples_top100.head()

In [ ]:
countSharedPNP434samples_top100=pd.DataFrame(countSharedPNP434samples_top100)
countSharedPNP434samples_top100=countSharedPNP434samples_top100.rename(columns={'aminoAcid':'n_shared_samples'})
countSharedPNP434samples_top100.head()

#### calculate the percentage of public sequences among the top100 sequences lists in the cohort:

In [ ]:
public_top100=countSharedPNP434samples_top100[countSharedPNP434samples_top100['n_shared_samples']>1]

In [ ]:
print len(public_top100)

In [ ]:
private_top100=countSharedPNP434samples_top100[countSharedPNP434samples_top100['n_shared_samples']==1]

In [ ]:
print len(private_top100)

In [ ]:
# the percent of public sequences in the cohort is 10.3%:
float(len(public_top100)*100)/(len(private_top100)+len(public_top100))

# public feature calculations:

## add number of shared samples to each uniqueAA sequence:

In [ ]:
public.head()

In [ ]:
countSharedPNP434samples['IsPublic']=np.where(countSharedPNP434samples['n_shared_samples']>1,1,0)

In [ ]:
All434samples_UniqueAAdfs_withCounting=pd.merge(All434samples_UniqueAAdfs_toMerge,countSharedPNP434samples,how='left', left_index=True, right_index=True)

In [ ]:
All434samples_UniqueAAdfs_withCounting.head()

In [ ]:
#save merged df to file:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PublicSeqAnalysis/All434samples_UniqueAAdfs_withCounting' 
All434samples_UniqueAAdfs_withCounting.to_pickle(file1)


## development of public feature calculations:

In [ ]:
sample_name='BD1'

sampleSequences=All434samples_UniqueAAdfs_withCounting[All434samples_UniqueAAdfs_withCounting['Sample']==sample_name]

In [ ]:
nTotal=len(sampleSequences)
nPublic=len(sampleSequences[sampleSequences['n_shared_samples']>1])
nSahredMore5=len(sampleSequences[sampleSequences['n_shared_samples']>5])
nSahredMore100=len(sampleSequences[sampleSequences['n_shared_samples']>100])

percPublic=round(float(nPublic)*100/nTotal,2)
percPublic5=round(float(nSahredMore5)*100/nTotal,2)
percPublic100=round(float(nSahredMore100)*100/nTotal,2)

print percPublic,percPublic5,percPublic100

## functions to calculate sharingFeatures for a cohort

In [ ]:
def gen_uniqueAAlists_forcohort(data_folder,cohortName):
    #get samples filenames:
    print 'getting samples names...'
    sample_dfs_folder='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/%s/SamplesForAnalysis' %data_folder
    sample_filenames = [f for f in listdir(sample_dfs_folder) if isfile(join(sample_dfs_folder, f))]
    sample_filenames=[f.strip('.tsv') for f in sample_filenames]
    
    
    #generate uniqueAA lists for each sample:
    print 'generating uniqueAA lists for each sample...'
    uniqueAAdfs_folder='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/%s/PublicSeqAnalysis/uniqueAAdfs/' %data_folder
    if not isdir(uniqueAAdfs_folder):
        mkdir(uniqueAAdfs_folder)

    for n,sample_name in enumerate(sample_filenames):    
            print  n,sample_name
            gen_uniqueAAdf(sample_name,data_folder)
            
    #concatenate all lists into one list for the wholecohort:
    print 'concatenating all uniqueAA lists...'
    uniqueAAdfs_folder='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/%s/PublicSeqAnalysis/uniqueAAdfs/' %data_folder
    AllSamples_UniqueAAdfs=concat_summarizing_dfs(uniqueAAdfs_folder)
    
    #get sub-dataframe to enable merging with counting DF:
    AllSamples_UniqueAAdfs_toMerge= AllSamples_UniqueAAdfs
    AllSamples_UniqueAAdfs_toMerge.columns =  AllSamples_UniqueAAdfs_toMerge.columns.get_level_values(0)
    AllSamples_UniqueAAdfs_toMerge.head()
    
    # count in how many samples each sequence appears?
    print 'counting in how many samples each sequence appears... (long!)'
    countSharedSamples= AllSamples_UniqueAAdfs_toMerge.index.value_counts()
    
    #save counting to file:
    print 'saving counting into file'
    file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/%s/PublicSeqAnalysis/countSharedSamples_%s' %(data_folder,cohortName) 
    countSharedSamples.to_pickle(file1)
    
    #merging counting information into the list of uniqueAA sequence of the cohort:
    print 'merging counting information into the list of uniqueAA sequence of the cohort...'
    
    countSharedSamples['IsPublic']=np.where(countSharedSamples['n_shared_samples']>1,1,0)
    AllSamples_UniqueAAdfs_withCounting=pd.merge(AllSamples_UniqueAAdfs_toMerge,countSharedSamples,how='left', left_index=True, right_index=True)
    
    #save merged df to file:
    print 'saving cohort uniqueAA list with counting information into file...'
    file2='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/%s/PublicSeqAnalysis/AllSamples_UniqueAAdfs_withCounting_%s' %(data_folder,cohortName)
    AllSamples_UniqueAAdfs_withCounting.to_pickle(file2)

    
    

In [ ]:
#this function calculates sharing features per sample, based on a 'uniqueAAdfWithCounting' dataframe.
#to calculate this required df, use 'gen_uniqueAAlists_forcohort'

def gen_SharingFeatures(data_folder,sample_name):

# make sure that uniqueAAdfWithCounting was called before the function is called. it is possible to add it as an input to the function)
    
    
    file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/%s/descriptiveStatsSamplesForAnalysis/Total/SharingFeatures/%s' %(data_folder,sample_name) 
    dfs_folder='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/%s/descriptiveStatsSamplesForAnalysis/Total/SharingFeatures' %data_folder
    if not isdir(dfs_folder):
        mkdir(dfs_folder) 
    files=[f for f in listdir(dfs_folder) if isfile(join(dfs_folder, f))]
    
    if sample_name not in files:
        print('calculating sharing features...')
    
        sampleSequences=uniqueAAdfWithCounting[uniqueAAdfWithCounting['Sample']==sample_name]
        nTotal=len(sampleSequences)
        nPublic=len(sampleSequences[sampleSequences['n_shared_samples']>1])
        nSahredMore5=len(sampleSequences[sampleSequences['n_shared_samples']>5])
        nSahredMore100=len(sampleSequences[sampleSequences['n_shared_samples']>100])

        percPublic=round(float(nPublic)*100/nTotal,2)
        percPublic5=round(float(nSahredMore5)*100/nTotal,2)
        percPublic100=round(float(nSahredMore100)*100/nTotal,2)

        SharingFeaturesDF=pd.DataFrame() #generate empty dataframe
        SharingFeaturesDF.loc[0,'Sample']=sample_name
        SharingFeaturesDF.loc[0,'percPublic']=percPublic
        SharingFeaturesDF.loc[0,'percPublic5']=percPublic5
        SharingFeaturesDF.loc[0,'percPublic100']=percPublic100

        SharingFeaturesDF.to_pickle(file1)
    else:
        print('found sharing features for this sample...')
    
    
    
    


## calculate sharing features:

In [ ]:
data_folder='TCR_real_data'
dfs_folder='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/%s/SamplesForAnalysis' %data_folder
filenames = [f for f in listdir(dfs_folder) if isfile(join(dfs_folder, f))]
filenames=[f.strip('.tsv') for f in filenames]
print len(filenames)

In [ ]:
uniqueAAdfWithCounting=All434samples_UniqueAAdfs_withCounting

for n,sample_name in enumerate(filenames):    
    print  n,sample_name
    gen_SharingFeatures(data_folder,sample_name)

In [ ]:
sharing_folder='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/%s/descriptiveStatsSamplesForAnalysis/Total/SharingFeatures' %data_folder
SharingFeatures=concat_summarizing_dfs(sharing_folder)
SharingFeatures.describe()

# generate publicNmore matrices

## load files:

In [ ]:

file2='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PublicSeqAnalysis/All434samples_UniqueAAdfs_withCounting'
AllSamples_UniqueAAdfs_withCounting=pd.read_pickle(file2)

In [ ]:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PublicSeqAnalysis/countSharedPNP434samples'
countSharedPNP434samples=pd.read_pickle(file1)

## merge the list of uniqueAA sequences per sample with the total list of shared seqeunces

In [ ]:
AllSamples_UniqueAAdfs_withCounting.sort_values(by='n_shared_samples',ascending=False).head()

In [ ]:
# this function merge the list of uniqueAA sequences per sample with the total list of shared seqeunces (which are shared among
# more than a threshold number of samples, defined below as 'nShared_cutoff'):

def gen_addPublicInfo_forSample(sample_name,publicNmoreDF):
    sampleSequences=AllSamples_UniqueAAdfs_withCounting[AllSamples_UniqueAAdfs_withCounting['Sample']==sample_name]
    sampleSequences=sampleSequences['count (templates)']
    sampleSequences=pd.DataFrame(sampleSequences)
    
    publicNmoreDF=pd.merge(publicNmoreDF,sampleSequences,how='left',left_index=True,right_index=True)
    publicNmoreDF=publicNmoreDF.rename(columns={'count (templates)':sample_name})
    
    return publicNmoreDF


## generate binary matrix for public sequence presence in each sample
'public' defintion depends on the number of required minimal number of samples sharing the same sequence (nShared_cutoff)

In [ ]:
#get the relevant samples for the cohort:

data_folder='TCR_real_data'
dfs_folder='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/%s/SamplesForAnalysis' %data_folder
filenames = [f for f in listdir(dfs_folder) if isfile(join(dfs_folder, f))]
filenames=[f.strip('.tsv') for f in filenames]
print len(filenames)

In [ ]:
#for each sample, add column to the matrix indicating presence/ansence of the shared sequence in the sample:

countSharedSamples=countSharedPNP434samples                                 #define counting df
# AllSamples_UniqueAAdfs_withCounting=All434samples_UniqueAAdfs_withCounting    #define uniqueAA df

nShared_cutoff=50 #define the lower thereshold of shared samples per sequence to generate the public matrix:

publicNmoreDF=countSharedSamples[countSharedSamples['n_shared_samples']>nShared_cutoff] #define the list of shared sequences in the
                                                                                        #cohort, based on the nShared_cutoff value
# publicNmoreDF=publicNmoreDF.drop('IsPublic',axis=1)
print 'the number of uniqueAA sequences shared by more than %s samples is %s' %(nShared_cutoff,len(publicNmoreDF))

for n,sample_name in enumerate(filenames):  
#     if n<4:
        print  n,sample_name
        publicNmoreDF=gen_addPublicInfo_forSample(sample_name,publicNmoreDF)


print 'now converting counts to binary indications...'
for column in publicNmoreDF.columns.values[1:]:
    publicNmoreDF[column]=np.where(publicNmoreDF[column]>0,1,0)
print 'DONE!'

In [ ]:
#check the correctness of the matrix: if the sum of all columns except for the n_shared_samples is similar to the n_shared_samples
#then the matrix is correct:

columnsToSum=[column for column in publicNmoreDF.columns.values if 'BD' in column]
publicNmoreDF['sum']=publicNmoreDF[columnsToSum].sum(axis=1)
publicNmoreDF['ratio']=publicNmoreDF['n_shared_samples']/publicNmoreDF['sum']

problem=publicNmoreDF[publicNmoreDF['ratio']!=1]
problem


In [ ]:
#drop unecessary columns,transform and save:
publicNmoreDF_toSave=publicNmoreDF.drop(['n_shared_samples','sum','ratio'],axis=1)
publicNmoreDF_toSave=publicNmoreDF_toSave.T
file3='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/%s/PublicSeqAnalysis/sharingMatrix_moreThan%s_%sSamples' %(data_folder,
                                                                                                                  nShared_cutoff,
                                                                                                                 len(filenames))
publicNmoreDF_toSave.to_pickle(file3)

In [ ]:
publicNmoreDF_toSave.head()

In [ ]:
publicNmoreDF_toSave.shape

In [ ]:
publicNmoreDF_toSave.sum()[-5:]

## generate relative abundance matrix for public sequence presence in each sample
'public' defintion depends on the number of required minimal number of samples sharing the same sequence (nShared_cutoff)

*** this section is a repat of the former one, but this time without the binary transformation! ***

### loading files:

In [ ]:
# load counting:
file2='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PublicSeqAnalysis/countSharedPNP434samples' 
countSharedPNP434samples=pd.read_pickle(file2)

In [ ]:
#load:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PublicSeqAnalysis/All434samples_UniqueAAdfs_withCounting' 
All434samples_UniqueAAdfs_withCounting=pd.read_pickle(file1)

In [ ]:
#get the relevant samples for the cohort:

data_folder='TCR_real_data'
dfs_folder='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/%s/SamplesForAnalysis' %data_folder
filenames = [f for f in listdir(dfs_folder) if isfile(join(dfs_folder, f))]
filenames=[f.strip('.tsv') for f in filenames]
print len(filenames)

In [ ]:
# this function merge the list of uniqueAA sequences per sample with the total list of shared seqeunces (which are shared among
# more than a threshold number of samples, defined below as 'nShared_cutoff')

##### this function includes transformation of the template counts into relative abundance ####

def gen_addPublicInfo_forSample_withRAcalc(sample_name,publicNmoreDF_RA):
    sampleSequences=AllSamples_UniqueAAdfs_withCounting[AllSamples_UniqueAAdfs_withCounting['Sample']==sample_name]
    sampleSequences=pd.DataFrame(sampleSequences['count (templates)'])
    sampleSequences['RA']=sampleSequences['count (templates)']/sampleSequences['count (templates)'].sum()
    sampleSequences=sampleSequences.drop('count (templates)',axis=1)
    print 'sanity check: sample RA total is %s' %sampleSequences['RA'].sum()
    
    publicNmoreDF_RA=pd.merge(publicNmoreDF_RA,sampleSequences,how='left',left_index=True,right_index=True)
    publicNmoreDF_RA=publicNmoreDF_RA.rename(columns={'RA':sample_name})
    
    return publicNmoreDF_RA


### generate relative  abundance matrix for public sequence presence in each sample
'public' defintion depends on the number of required minimal number of samples sharing the same sequence (nShared_cutoff)

In [ ]:
#get the relevant samples for the cohort:

data_folder='TCR_real_data'
dfs_folder='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/%s/SamplesForAnalysis' %data_folder
filenames = [f for f in listdir(dfs_folder) if isfile(join(dfs_folder, f))]
filenames=[f.strip('.tsv') for f in filenames]
print len(filenames)

In [ ]:
#for each sample, add column to the matrix indicating presence/ansence of the shared sequence in the sample:

countSharedSamples=countSharedPNP434samples                                 #define counting df
AllSamples_UniqueAAdfs_withCounting=All434samples_UniqueAAdfs_withCounting    #define uniqueAA df

nShared_cutoff=50 #define the lower thereshold of shared samples per sequence to generate the public matrix:

publicNmoreDF_RA=countSharedSamples[countSharedSamples['n_shared_samples']>nShared_cutoff] #define the list of shared sequences in the
                                                                                        #cohort, based on the nShared_cutoff value
# publicNmoreDF_RA=publicNmoreDF_RA.drop('IsPublic',axis=1)
print 'the number of uniqueAA sequences shared by more than %s samples is %s' %(nShared_cutoff,len(publicNmoreDF_RA))

for n,sample_name in enumerate(filenames):  
#     if n<4:
        print  n,sample_name
        publicNmoreDF_RA=gen_addPublicInfo_forSample_withRAcalc(sample_name,publicNmoreDF_RA)

print 'DONE!'

In [ ]:
publicNmoreDF_RA

In [ ]:
#drop unecessary columns,transform and save:
publicNmoreDF_RA_toSave=publicNmoreDF_RA.drop('n_shared_samples',axis=1)
publicNmoreDF_RA_toSave=publicNmoreDF_RA_toSave.T
publicNmoreDF_RA_toSave=publicNmoreDF_RA_toSave.fillna(0)
file3='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/%s/PublicSeqAnalysis/sharingMatrix_moreThan%s_%sSamples_RA' %(data_folder,
                                                                                                                  nShared_cutoff,
                                                                                                                 len(filenames))
publicNmoreDF_RA_toSave.to_pickle(file3)

In [ ]:
publicNmoreDF_RA_toSave.head()

### transform publicNmoreDF_RA to log2 scale and save:

In [ ]:
short=publicNmoreDF_RA_toSave.iloc[0:5,0:5]
short

replace -inf values with -1000:

In [ ]:
inf=short_log2scale.iloc['BD438','CASSLGETQYF']

In [ ]:
short_log2scale = short.applymap(np.log2)
short_log2scale=short_log2scale.replace(inf,-1000)
short_log2scale.head()

In [ ]:
publicNmoreDF_RA_toSave_log2scale = publicNmoreDF_RA_toSave.applymap(np.log2)

In [ ]:
publicNmoreDF_RA_toSave_log2scale.head()

change from here:

In [ ]:
publicNmoreDF_RA_toSave_log2scale=publicNmoreDF_RA_toSave_log2scale.replace(inf,-1000)
publicNmoreDF_RA_toSave_log2scale.head()

In [ ]:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/%s/PublicSeqAnalysis/sharingMatrix_moreThan%s_%sSamples_RA_log2scale' %(data_folder,
                                                                                                                  nShared_cutoff,
                                                                                                                 len(filenames))
publicNmoreDF_RA_toSave_log2scale.to_pickle(file1)

## generate binary matrix for public sequence presence in each sample
'public' defintion depends on the number of required minimal number of samples sharing the same sequence (nShared_cutoff)

In [ ]:
#get the relevant samples for the cohort:

data_folder='TCR_real_data'
dfs_folder='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/%s/SamplesForAnalysis' %data_folder
filenames = [f for f in listdir(dfs_folder) if isfile(join(dfs_folder, f))]
filenames=[f.strip('.tsv') for f in filenames]
print len(filenames)

In [ ]:
#for each sample, add column to the matrix indicating presence/ansence of the shared sequence in the sample:

countSharedSamples=countSharedPNP434samples                                 #define counting df
AllSamples_UniqueAAdfs_withCounting=All434samples_UniqueAAdfs_withCounting    #define uniqueAA df

nShared_cutoff=2 #define the lower thereshold of shared samples per sequence to generate the public matrix:

publicNmoreDF=countSharedSamples[countSharedSamples['n_shared_samples']>nShared_cutoff] #define the list of shared sequences in the
                                                                                        #cohort, based on the nShared_cutoff value
publicNmoreDF=publicNmoreDF.drop('IsPublic',axis=1)
print 'the number of uniqueAA sequences shared by more than %s samples is %s' %(nShared_cutoff,len(publicNmoreDF))

for n,sample_name in enumerate(filenames):  
#     if n<4:
        print  n,sample_name
        publicNmoreDF=gen_addPublicInfo_forSample(sample_name,publicNmoreDF)


print 'now converting counts to binary indications...'
for column in publicNmoreDF.columns.values[1:]:
    publicNmoreDF[column]=np.where(publicNmoreDF[column]>0,1,0)
print 'DONE!'

In [ ]:
#check the correctness of the matrix: if the sum of all columns except for the n_shared_samples is similar to the n_shared_samples
#then the matrix is correct:

columnsToSum=[column for column in publicNmoreDF.columns.values if 'BD' in column]
publicNmoreDF['sum']=publicNmoreDF[columnsToSum].sum(axis=1)
publicNmoreDF['ratio']=publicNmoreDF['n_shared_samples']/publicNmoreDF['sum']

problem=publicNmoreDF[publicNmoreDF['ratio']!=1]
problem


In [ ]:
#drop unecessary columns,transform and save:
publicNmoreDF_toSave=publicNmoreDF.drop(['n_shared_samples','sum','ratio'],axis=1)
publicNmoreDF_toSave=publicNmoreDF_toSave.T
file3='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/%s/PublicSeqAnalysis/sharingMatrix_moreThan%s_%sSamples' %(data_folder,
                                                                                                                  nShared_cutoff,
                                                                                                                 len(filenames))
publicNmoreDF_toSave.to_pickle(file3)

In [ ]:
publicNmoreDF_toSave.head()

In [ ]:
publicNmoreDF_toSave.shape

In [ ]:
publicNmoreDF_toSave.sum()[:5]

# generate PC matrices as features:

## PCA to shared by more than 1 sample-RA file:

In [ ]:
nShared_cutoff=50  
file3='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PublicSeqAnalysis/sharingMatrix_moreThan1_434Samples_RA'
sharingMatrix_moreThan1_434Samples_RA=pd.read_pickle(file3)
sharingMatrix_moreThan1_434Samples_RA.head()

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=100)
pca.fit(sharingMatrix_moreThan1_434Samples_RA)

In [ ]:
print(pca.components_)

In [ ]:
print(pca.explained_variance_)

In [ ]:
# plot principal components
X_pca = pca.transform(sharingMatrix_moreThan1_434Samples_RA)
plt.scatter(X_pca[:, 0], X_pca[:, 1], alpha=0.2)
# plt.axis('equal')
# plt.xlabel('component 1')
# plt.ylabel('component 2')
plt.title('principal components')
# plt.xlim(-20,20)
# plt.ylim(-20,20)


plt.show()

# fig.savefig('figures/05.09-PCA-rotation.png')

## PCA to shared by more than 10 sample:

In [ ]:
file3='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PublicSeqAnalysis/sharingMatrix_moreThan10_434Samples'
sharingMatrix_moreThan10_434Samples=pd.read_pickle(file3)
sharingMatrix_moreThan10_434Samples.head()

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=100)
pca.fit(sharingMatrix_moreThan10_434Samples.T)

In [ ]:
print(pca.components_)

In [ ]:
print len(pca.components_[0])

In [ ]:
print(pca.explained_variance_)

In [ ]:
# plot principal components
X_pca = pca.transform(sharingMatrix_moreThan10_434Samples.T)
plt.scatter(X_pca[:, 0], X_pca[:, 1], alpha=0.2)
# plt.axis('equal')
# plt.xlabel('component 1')
# plt.ylabel('component 2')
plt.title('principal components')
# plt.xlim(-20,20)
# plt.ylim(-20,20)


plt.show()

### generate PCA df as a feature DF

In [ ]:
PCAdf=pd.DataFrame()
for n,PC in enumerate(pca.components_):
    colName='PC'+str(n)
#     print colName
    PCAdf[colName]=PC
    PCAdf=PCAdf.set_index(sharingMatrix_moreThan10_434Samples.index)
    

In [ ]:
PCAdf

## define general function for PCA analysis of sharing matrices:

In [ ]:
'''
this function performs PCA analysis to a specific sequence matrix defined by nSharedCutoff and isRA
it outputs a PC1+PC2 plot, the percentage explained by the first PCs, and a df detailing the first 100PCs which can then be used
a feature

input:
nSharedCutoff-1/2/5/10
isRA- True/False
nPCs - the number of PCs requested



'''

def PCAtoSharedSequenceMatrix(nSharedCutoff, isRA,nPCs):

    #load sharing matrix:

    if isRA:
        RA='_RA'
    else:
        RA=''

    file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PublicSeqAnalysis/sharingMatrix_moreThan%s_434Samples%s' %(nSharedCutoff,RA)
    sharingMatrix=pd.read_pickle(file1)
    pd.read_pickle(file1)
    
    # perform PCA

    from sklearn.decomposition import PCA
    pca = PCA(n_components=100)
    pca.fit(sharingMatrix.T) #it is very important to transform the matrix!
    print 'percent explained by PC1 is %s' %pca.explained_variance_[0]
    print 'percent explained by PC2 is %s' %pca.explained_variance_[1]
    
    # plot principal components
    fig1=plt.figure (figsize=(6,6))
    X_pca = pca.transform(sharingMatrix.T)
    plt.scatter(X_pca[:, 0], X_pca[:, 1], alpha=0.2)
    plt.axis('equal')
    plt.xlabel('PC0')
    plt.ylabel('PC1')
    plt.title('first2PCs_sharingMatrix_moreThan%s_434Samples%s' %(nSharedCutoff,RA),fontsize=12)
    
    file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PublicSeqAnalysis/PCAplots/first2PCs_of%sPCs_sharingMatrix_moreThan%s_434Samples%s' %(nPCs,nSharedCutoff,RA)
    fig1.savefig(file1, dpi=200)
    
    # generate PCA df as a feature DF

    PCAdf=pd.DataFrame()
    for n,PC in enumerate(pca.components_):
        colName='PC'+str(n)
    #     print colName
        PCAdf[colName]=PC
        PCAdf=PCAdf.set_index(sharingMatrix.index)
        
    file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PublicSeqAnalysis/sharingMatrix_moreThan%s_434Samples%s_%sPCs' %(nSharedCutoff,RA,nPCs)
    PCAdf.to_pickle(file1)




# run PCA analysis on different sharing sequence matrices

In [ ]:
nList=[1,2,5,10]
isRAlist=[True] # it seems that the PCA analysis on the RA matrix is less efficient
# isRAlist=[True,False]
nPCsList=[5,100]

for nSharedCutoff in nList:
    for isRA in isRAlist:
        for nPCs in nPCsList:
            print nSharedCutoff, isRA, nPCs
            PCAtoSharedSequenceMatrix(nSharedCutoff, isRA,nPCs)

conclusiongs:
nPCs has no meaning
PCA analysis for RA matrices is much worse than the binary matrices
PCA analysis is better for low Ns

chose binay matrices for 1 and 10

## 1.2.18: run PCA analysis on moreThan50:

In [ ]:
nList=[50]
isRAlist=[False] # it seems that the PCA analysis on the RA matrix is less efficient
# isRAlist=[True,False]
nPCsList=[1,5,100]

for nSharedCutoff in nList:
    for isRA in isRAlist:
        for nPCs in nPCsList:
            print nSharedCutoff, isRA, nPCs
            PCAtoSharedSequenceMatrix(nSharedCutoff, isRA,nPCs)